# Basics

The `fairness` package does a few nice things for you. For example, it provides a standardized access to different ways of preprocessing a dataset:

In [ ]:
#import sys
#!{sys.executable} -m pip install fairness
#!{sys.executable} -m pip install ggplot

In [1]:
import fairness.benchmark

Available algorithms:
  SVM
  GaussianNB
  LR
  DecisionTree
  Kamishima
  Calders
  ZafarBaseline
  ZafarFairness
  ZafarAccuracy
  Kamishima-accuracy
  Kamishima-DIavgall
  Feldman-SVM
  Feldman-GaussianNB
  Feldman-LR
  Feldman-DecisionTree
  Feldman-SVM-DIavgall
  Feldman-SVM-accuracy
  Feldman-GaussianNB-DIavgall
  Feldman-GaussianNB-accuracy


In [2]:
from fairness.data.objects.Ricci import Ricci
from fairness.data.objects.ProcessedData import ProcessedData


In [3]:
ricci = ProcessedData(Ricci())

In [4]:
ricci.get_dataframe("original")

,Position,Oral,Written,Race,Combine,Class
0,Captain,89.52,95,W,92.808,1
1,Captain,80.00,95,W,89.000,1
2,Captain,82.38,87,W,85.152,1
3,Captain,88.57,76,W,81.028,1
4,Captain,76.19,84,W,80.876,1
...,...,...,...,...,...,...
113,Lieutenant,40.83,64,H,54.732,0
114,Lieutenant,52.08,56,B,54.432,0
115,Lieutenant,48.33,58,H,54.132,0
116,Lieutenant,52.92,49,B,50.568,0


In [5]:
ricci.get_dataframe("numerical-binsensitive")

,Oral,Written,Race,Combine,Class,Position_Captain,Position_Lieutenant
0,89.52,95,1,92.808,1,1,0
1,80.00,95,1,89.000,1,1,0
2,82.38,87,1,85.152,1,1,0
3,88.57,76,1,81.028,1,1,0
4,76.19,84,1,80.876,1,1,0
...,...,...,...,...,...,...,...
113,40.83,64,0,54.732,0,0,1
114,52.08,56,0,54.432,0,0,1
115,48.33,58,0,54.132,0,0,1
116,52.92,49,0,50.568,0,0,1


## Necessary imports
We'll be using these throughout the tutorial.

In [6]:
# We'll suppress warnings because both altair and sklearn are
# emitting lots of them, and they're annoying in a demo setting.

import warnings
warnings.filterwarnings("ignore")

import altair as alt
# Ask Altair to produce output that works on Jupyter Notebook
alt.renderers.enable('notebook')

RendererRegistry.enable('notebook')

In [7]:
import fairness
import fairness.benchmark

In order to run specific algorithms and datasets, pass them as parameters to `fairness.benchmark.run`. Do note that if you don't pass these parameters, `fairness` will run all available algorithms on all available datasets, which will take a *very* long time (about a week on a single processor in our machines)!

In [8]:
fairness.benchmark.run(algorithm=["LR", "Feldman-LR"], dataset=["ricci"])

Datasets: '['ricci']'

Evaluating dataset:ricci
Sensitive attribute:Race
    Algorithm: LR
       supported types: {'numerical', 'numerical-binsensitive'}
    Algorithm: Feldman-LR
       supported types: {'numerical', 'numerical-binsensitive'}
Results written to:
    C:\Users\hatta\.fairness\results\ricci_Race_numerical.csv
    C:\Users\hatta\.fairness\results\ricci_Race_numerical-binsensitive.csv


You can then access the results per dataset, sensitive attribute, and preprocessing option:

In [9]:
ricci_Race = fairness.get_dataset_by_name("ricci").get_results_data_frame("Race", "numerical-binsensitive")

In [10]:
# So many measures!
list(ricci_Race.columns.values)

['algorithm',
 'params',
 'run-id',
 '0-FNR',
 '0-FNR_over_1-FNR',
 '0-FPR',
 '0-FPR_over_1-FPR',
 '0-TNR',
 '0-TNR_over_1-TNR',
 '0-TPR',
 '0-TPR_over_1-TPR',
 '0-accuracy',
 '0-accuracy_over_1-accuracy',
 '0-calibration+',
 '0-calibration+_over_1-calibration+',
 '0-calibration-',
 '0-calibration-_over_1-calibration-',
 '1-FNR',
 '1-FPR',
 '1-TNR',
 '1-TPR',
 '1-accuracy',
 '1-calibration+',
 '1-calibration-',
 'BCR',
 'CV',
 'DIavgall',
 'DIbinary',
 'MCC',
 'Race-FNR',
 'Race-FNRDiff',
 'Race-FNRRatio',
 'Race-FPR',
 'Race-FPRDiff',
 'Race-FPRRatio',
 'Race-TNR',
 'Race-TNRDiff',
 'Race-TNRRatio',
 'Race-TPR',
 'Race-TPRDiff',
 'Race-TPRRatio',
 'Race-accuracy',
 'Race-accuracyDiff',
 'Race-accuracyRatio',
 'Race-calibration+',
 'Race-calibration+Diff',
 'Race-calibration+Ratio',
 'Race-calibration-',
 'Race-calibration-Diff',
 'Race-calibration-Ratio',
 'TNR',
 'TPR',
 'accuracy',
 'diff:1-FNRto0-FNR',
 'diff:1-FPRto0-FPR',
 'diff:1-TNRto0-TNR',
 'diff:1-TPRto0-TPR',
 'diff:1-accur

To show a final example of what you can get out of `fairness`, we produce a simple plot of accuracy vs (binarized) disparate impact where color is used to differentiate algorithms:

In [11]:
ricci_Race = ricci_Race[ricci_Race.algorithm.str.contains("LR")]

In [12]:
ricci_Race

,algorithm,params,run-id,0-FNR,0-FNR_over_1-FNR,0-FPR,0-FPR_over_1-FPR,0-TNR,0-TNR_over_1-TNR,0-TPR,...,TNR,TPR,accuracy,diff:1-FNRto0-FNR,diff:1-FPRto0-FPR,diff:1-TNRto0-TNR,diff:1-TPRto0-TPR,diff:1-accuracyto0-accuracy,diff:1-calibration+to0-calibration+,diff:1-calibration-to0-calibration-
0,LR,NaN,0,0.0,1.0,0.000000,1.000000,1.000000,1.000000,1.0,...,1.000000,1.000000,1.000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,LR,NaN,1,0.0,1.0,0.000000,0.000000,1.000000,1.100000,1.0,...,0.960000,1.000000,0.975,1.000000,0.909091,1.090909,1.000000,1.043478,1.076923,1.000000
2,LR,NaN,2,0.0,1.0,0.000000,0.000000,1.000000,1.111111,1.0,...,0.947368,1.000000,0.975,1.000000,0.900000,1.100000,1.000000,1.037037,1.055556,1.000000
3,LR,NaN,3,0.0,1.0,0.000000,1.000000,1.000000,1.000000,1.0,...,1.000000,1.000000,1.000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
4,LR,NaN,4,0.0,1.0,0.000000,1.000000,1.000000,1.000000,1.0,...,1.000000,1.000000,1.000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
5,LR,NaN,5,0.0,1.0,0.000000,1.000000,1.000000,1.000000,1.0,...,1.000000,1.000000,1.000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
6,LR,NaN,6,0.0,1.0,0.000000,0.000000,1.000000,1.090909,1.0,...,0.954545,1.000000,0.975,1.000000,0.916667,1.083333,1.000000,1.040000,1.071429,1.000000
7,LR,NaN,7,0.0,1.0,0.000000,0.000000,1.000000,1.142857,1.0,...,0.950000,1.000000,0.975,1.000000,0.875000,1.125000,1.000000,1.043478,1.062500,1.000000
8,LR,NaN,8,0.0,1.0,0.000000,1.000000,1.000000,1.000000,1.0,...,1.000000,1.000000,1.000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
9,LR,NaN,9,0.0,0.0,0.000000,1.000000,1.000000,1.000000,1.0,...,1.000000,0.941176,0.975,0.909091,1.000000,1.000000,1.090909,1.050000,1.000000,0.900000


In [13]:
alt.renderers.enable('default')
alt.Chart(ricci_Race).mark_point().encode(
    x='accuracy',
    y='DIbinary',
    color='algorithm'
)

alt.Chart(...)

## Fairness Project Comparison Part (instead of the zip file, these are original py files called from the original library)
from https://github.com/algofairness/fairness-comparison
The github page states: "The below instructions are still in the process of being updated to work with the new pip install-able version."

In [4]:
from fairness.benchmark import run

Available algorithms:
  SVM
  GaussianNB
  LR
  DecisionTree
  Kamishima
  Calders
  ZafarBaseline
  ZafarFairness
  ZafarAccuracy
  Kamishima-accuracy
  Kamishima-DIavgall
  Feldman-SVM
  Feldman-GaussianNB
  Feldman-LR
  Feldman-DecisionTree
  Feldman-SVM-DIavgall
  Feldman-SVM-accuracy
  Feldman-GaussianNB-DIavgall
  Feldman-GaussianNB-accuracy


In [ ]:
#does not work, gives a lot of errors and I have to stop exacution manually:
#'IndexError: too many indices for array' and 'Failed: too many indices for array for several algorithms in the library'
run()

Datasets: '['ricci', 'adult', 'german', 'propublica-recidivism', 'propublica-violent-recidivism']'

Evaluating dataset:ricci
Sensitive attribute:Race
    Algorithm: SVM
       supported types: {'numerical', 'numerical-binsensitive'}
    Algorithm: GaussianNB
       supported types: {'numerical', 'numerical-binsensitive'}
    Algorithm: LR
       supported types: {'numerical', 'numerical-binsensitive'}
    Algorithm: DecisionTree
       supported types: {'numerical', 'numerical-binsensitive'}
    Algorithm: Kamishima
       supported types: {'numerical-binsensitive'}


C:\Users\hatta\anaconda3\lib\site-packages\fairness\algorithms\kamishima\KamishimaAlgorithm.py:99: UserWarning: loadtxt: Empty input file: "C:\Users\hatta\AppData\Local\Temp\tmpsx4dtaun"
  m = numpy.loadtxt(output_name)
Traceback (most recent call last):
  File "C:\Users\hatta\anaconda3\lib\site-packages\fairness\benchmark.py", line 66, in run
    run_eval_alg(algorithm, train, test, dataset_obj, processed_dataset,
  File "C:\Users\hatta\anaconda3\lib\site-packages\fairness\benchmark.py", line 107, in run_eval_alg
    run_alg(algorithm, train, test, dataset, all_sensitive_attributes, single_sensitive,
  File "C:\Users\hatta\anaconda3\lib\site-packages\fairness\benchmark.py", line 145, in run_alg
    algorithm.run(train, test, class_attr, positive_val, all_sensitive_attributes,
  File "C:\Users\hatta\anaconda3\lib\site-packages\fairness\algorithms\kamishima\KamishimaAlgorithm.py", line 102, in run
    predictions = m[:,1]
IndexError: too many indices for array
Failed: too many indices f

Traceback (most recent call last):
  File "C:\Users\hatta\anaconda3\lib\site-packages\fairness\benchmark.py", line 66, in run
    run_eval_alg(algorithm, train, test, dataset_obj, processed_dataset,
  File "C:\Users\hatta\anaconda3\lib\site-packages\fairness\benchmark.py", line 107, in run_eval_alg
    run_alg(algorithm, train, test, dataset, all_sensitive_attributes, single_sensitive,
  File "C:\Users\hatta\anaconda3\lib\site-packages\fairness\benchmark.py", line 145, in run_alg
    algorithm.run(train, test, class_attr, positive_val, all_sensitive_attributes,
  File "C:\Users\hatta\anaconda3\lib\site-packages\fairness\algorithms\kamishima\KamishimaAlgorithm.py", line 102, in run
    predictions = m[:,1]
IndexError: too many indices for array
Failed: too many indices for array
C:\Users\hatta\anaconda3\lib\site-packages\fairness\algorithms\kamishima\KamishimaAlgorithm.py:99: UserWarning: loadtxt: Empty input file: "C:\Users\hatta\AppData\Local\Temp\tmpfd_wukpv"
  m = numpy.loadtxt(outp

    Algorithm: Calders
       supported types: {'numerical-binsensitive'}


Traceback (most recent call last):
  File "C:\Users\hatta\anaconda3\lib\site-packages\fairness\benchmark.py", line 66, in run
    run_eval_alg(algorithm, train, test, dataset_obj, processed_dataset,
  File "C:\Users\hatta\anaconda3\lib\site-packages\fairness\benchmark.py", line 107, in run_eval_alg
    run_alg(algorithm, train, test, dataset, all_sensitive_attributes, single_sensitive,
  File "C:\Users\hatta\anaconda3\lib\site-packages\fairness\benchmark.py", line 145, in run_alg
    algorithm.run(train, test, class_attr, positive_val, all_sensitive_attributes,
  File "C:\Users\hatta\anaconda3\lib\site-packages\fairness\algorithms\kamishima\CaldersAlgorithm.py", line 99, in run
    raise Exception("Training procedure failed")
Exception: Training procedure failed
Failed: Training procedure failed
Traceback (most recent call last):
  File "C:\Users\hatta\anaconda3\lib\site-packages\fairness\benchmark.py", line 66, in run
    run_eval_alg(algorithm, train, test, dataset_obj, processed_dat

...

Traceback (most recent call last):
  File "C:\Users\hatta\anaconda3\lib\site-packages\fairness\benchmark.py", line 66, in run
    run_eval_alg(algorithm, train, test, dataset_obj, processed_dataset,
  File "C:\Users\hatta\anaconda3\lib\site-packages\fairness\benchmark.py", line 107, in run_eval_alg
    run_alg(algorithm, train, test, dataset, all_sensitive_attributes, single_sensitive,
  File "C:\Users\hatta\anaconda3\lib\site-packages\fairness\benchmark.py", line 145, in run_alg
    algorithm.run(train, test, class_attr, positive_val, all_sensitive_attributes,
  File "C:\Users\hatta\anaconda3\lib\site-packages\fairness\algorithms\ParamGridSearch.py", line 36, in run
    best_predictions = self.find_best(all_predictions, train_df, test_df, class_attr,
  File "C:\Users\hatta\anaconda3\lib\site-packages\fairness\algorithms\ParamGridSearch.py", line 44, in find_best
    raise Exception(
Exception: No run in the parameter grid search succeeded - failing run of algorithm
Failed: No run in t

    Algorithm: Feldman-SVM
       supported types: {'numerical', 'numerical-binsensitive'}
    Algorithm: Feldman-GaussianNB
       supported types: {'numerical', 'numerical-binsensitive'}
    Algorithm: Feldman-LR
       supported types: {'numerical', 'numerical-binsensitive'}
    Algorithm: Feldman-DecisionTree
       supported types: {'numerical', 'numerical-binsensitive'}
    Algorithm: Feldman-SVM-DIavgall
       supported types: {'numerical', 'numerical-binsensitive'}
    Algorithm: Feldman-SVM-accuracy
       supported types: {'numerical', 'numerical-binsensitive'}
    Algorithm: Feldman-GaussianNB-DIavgall
       supported types: {'numerical', 'numerical-binsensitive'}
    Algorithm: Feldman-GaussianNB-accuracy
       supported types: {'numerical', 'numerical-binsensitive'}
Results written to:
    C:\Users\hatta\.fairness\results\ricci_Race_numerical.csv
    C:\Users\hatta\.fairness\results\ricci_Race_numerical-binsensitive.csv

Evaluating dataset:adult
Sensitive attribute:race

C:\Users\hatta\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\hatta\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\hatta\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\hatta\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall is ill-defined and being se

    Algorithm: GaussianNB
       supported types: {'numerical', 'numerical-binsensitive'}


C:\Users\hatta\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [16]:
#function to run all the preprocessing step to create different versions of the datassets
from fairness import preprocess

In [17]:
preprocess.main()

--- Processing dataset: ricci ---

-------------------
Balance statistics:

Class:
Class
0    62
1    56
dtype: int64

Sensitive Attribute:
Race
B    27
H    23
W    68
dtype: int64




Writing data to: data\preprocessed\ricci_original.csv
Writing data to: data\preprocessed\ricci_numerical.csv
Writing data to: data\preprocessed\ricci_numerical-binsensitive.csv
Writing data to: data\preprocessed\ricci_categorical-binsensitive.csv
--- Processing dataset: adult ---
Missing Data: 2399 rows removed from dataset adult

-------------------
Balance statistics:

Class:
income-per-year
<=50K    22654
>50K      7508
dtype: int64

Sensitive Attribute:
race
Amer-Indian-Eskimo      286
Asian-Pac-Islander      895
Black                  2817
Other                   231
White                 25933
dtype: int64


sex
Female     9782
Male      20380
dtype: int64




Writing data to: data\preprocessed\adult_original.csv
Writing data to: data\preprocessed\adult_numerical.csv
Writing data to: data\preproce

ERROR: Could not consume arg: -f
Usage: ipykernel_launcher.py -

For detailed information on this command, run:
  ipykernel_launcher.py - --help


FireExit: 2

In [20]:
#analysis part
from fairness import analysis
analysis.main()
#AttributeError: module 'pandas' has no attribute 'tslib' 
#the first error was regarding ggplot(?) and in connection to pandas lib, I had to change several things from the ggplot library's source code, 
#some of the recommendations was using plotnine instead of ggplot, but I do not wish to invest time in changing the plotting of this project.
#but then I got another error:TypeError: can only concatenate str (not "WindowsPath") to str 
#it is buggy? >> https://github.com/takluyver/bookbook/issues/19


Generating graphs for dataset:ricci
    type:original
Skipping graph containing no data:ricci_Race_original
Skipping measures: sex-TPR sex-calibration-
    type:numerical
DIbinary accuracy
Skipping measures: sex-TPR sex-calibration-
    type:numerical-binsensitive
DIbinary accuracy
Skipping measures: sex-TPR sex-calibration-
    type:categorical-binsensitive
Skipping graph containing no data:ricci_Race_categorical-binsensitive
Skipping measures: sex-TPR sex-calibration-

Generating graphs for dataset:adult
    type:original


TypeError: can only concatenate str (not "WindowsPath") to str

In [2]:
#problematic, because i cannot pass the algorithm name through main function, it runs fire() and fire function runs the function run(), which requires the algorithm's name
#when I pass the alg. name with the actual function run(), I get another error that is not related to me: TypeError: 'str' object cannot be interpreted as an integer
#in the source code (ProcessedData.py), i changed the variable num to int(num), then I got the current error about non-existing csv file.
from fairness import get_best_split
get_best_split.main(get_best_split.run('LR'))

    type:original
    type:numerical
    type:numerical-binsensitive
    type:categorical-binsensitive
    type:original
    type:numerical
    type:numerical-binsensitive
    type:categorical-binsensitive
    type:original
    type:numerical
    type:numerical-binsensitive
    type:categorical-binsensitive
results/LR-accuracy_adult_race_original.csv


FileNotFoundError: [Errno 2] File results/LR-accuracy_adult_race_original.csv does not exist: 'results/LR-accuracy_adult_race_original.csv'